In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
from tensorflow.contrib import slim
from tensorflow.contrib.slim import nets

In [2]:
# 字符集
CHAR_SET = [str(i) for i in "0123456789ABCDE"]
CHAR_SET_LEN = len(CHAR_SET)
print(CHAR_SET_LEN)
# 训练集大小
TRAIN_NUM = 44892
# 批次大小
BATCH_SIZE = 256
# 迭代次数
EPOCHES = 40
# 循环次数
LOOP_TIMES = EPOCHES*TRAIN_NUM//BATCH_SIZE
# tf文件
TFRECORD_TRAIN_FILE = 'captcha/tfrecords/train.tfrecord'

# 初始学习率
LEARNING_RATE = 0.0001

15


In [3]:
def read_and_decode(filename):
    tf_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(tf_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'image/encoded':tf.FixedLenFeature([], tf.string),
                                            'image/label0': tf.FixedLenFeature([], tf.int64),
                                            'image/label1': tf.FixedLenFeature([], tf.int64),
                                            'image/label2': tf.FixedLenFeature([], tf.int64),
                                            'image/label3': tf.FixedLenFeature([], tf.int64),
                                       })
    image = tf.decode_raw(features['image/encoded'], tf.uint8)
    image = tf.reshape(image,[224,224,1])
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)

    label0 = tf.cast(features['image/label0'], tf.int32)
    label1 = tf.cast(features['image/label1'], tf.int32)
    label2 = tf.cast(features['image/label2'], tf.int32)
    label3 = tf.cast(features['image/label3'], tf.int32)
    return image, label0, label1, label2, label3

In [4]:
image_tensor, label0_tensor, label1_tensor, label2_tensor, label3_tensor = read_and_decode(TFRECORD_TRAIN_FILE)

image_batch_tensor, label0_batch_tensor, label1_batch_tensor, label2_batch_tensor, label3_batch_tensor = tf.train.shuffle_batch(
    [image_tensor, label0_tensor, label1_tensor, label2_tensor, label3_tensor],
    batch_size=BATCH_SIZE,
    capacity=10000,
    min_after_dequeue=2000,
    num_threads=1000
)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [5]:
# 网络 
x = tf.placeholder(tf.float32,[None,224,224,1])
y0 = tf.placeholder(tf.int32,[None])
y1 = tf.placeholder(tf.int32,[None])
y2 = tf.placeholder(tf.int32,[None])
y3 = tf.placeholder(tf.int32,[None])
lr = tf.Variable(LEARNING_RATE, dtype=tf.float32,trainable=False)

with slim.arg_scope(nets.alexnet.alexnet_v2_arg_scope(weight_decay=0.0005)):
    outputs, end_points = nets.alexnet.alexnet_v2(x,CHAR_SET_LEN,is_training=True,dropout_keep_prob=1)
    #modify alexnet_v2
    fc7 = tf.get_default_graph().get_tensor_by_name("alexnet_v2/fc7/Relu:0")

    logits0 = tf.contrib.layers.conv2d(
            fc7,
            CHAR_SET_LEN, [1, 1],
            activation_fn=None,
            normalizer_fn=None,
            weights_initializer=tf.truncated_normal_initializer(0.0, 0.005),
            biases_initializer=tf.zeros_initializer(),
            scope='fc8-0')
    logits1 = tf.contrib.layers.conv2d(
            fc7,
            CHAR_SET_LEN, [1, 1],
            activation_fn=None,
            normalizer_fn=None,
            weights_initializer=tf.truncated_normal_initializer(0.0, 0.005),
            biases_initializer=tf.zeros_initializer(),
            scope='fc8-1')
    logits2 = tf.contrib.layers.conv2d(
            fc7,
            CHAR_SET_LEN, [1, 1],
            activation_fn=None,
            normalizer_fn=None,
            weights_initializer=tf.truncated_normal_initializer(0.0, 0.005),
            biases_initializer=tf.zeros_initializer(),
            scope='fc8-2')
    logits3 = tf.contrib.layers.conv2d(
            fc7,
            CHAR_SET_LEN, [1, 1],
            activation_fn=None,
            normalizer_fn=None,
            weights_initializer=tf.truncated_normal_initializer(0.0, 0.005),
            biases_initializer=tf.zeros_initializer(),
            scope='fc8-3')

logits0 = tf.squeeze(logits0, [1, 2], name='fc8-0/squeezed')
logits1 = tf.squeeze(logits1, [1, 2], name='fc8-1/squeezed')
logits2 = tf.squeeze(logits2, [1, 2], name='fc8-2/squeezed')
logits3 = tf.squeeze(logits3, [1, 2], name='fc8-3/squeezed')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
loss0 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y0,logits=logits0))
loss1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y1,logits=logits1))
loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y2,logits=logits2))
loss3 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y3,logits=logits3))

total_loss = (loss0 + loss1 + loss2 + loss3)/4.0
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(total_loss)

# 计算准确率
correct_pre0 = tf.cast(tf.equal(y0, tf.argmax(logits0, 1,output_type=tf.int32)),tf.float32)
accuracy0 = tf.reduce_mean(correct_pre0)

correct_pre1 = tf.cast(tf.equal(y1, tf.argmax(logits1, 1,output_type=tf.int32)),tf.float32)
accuracy1 = tf.reduce_mean(correct_pre1)

correct_pre2 = tf.cast(tf.equal(y2, tf.argmax(logits2, 1,output_type=tf.int32)),tf.float32)
accuracy2 = tf.reduce_mean(correct_pre2)

correct_pre3 = tf.cast(tf.equal(y3, tf.argmax(logits3, 1,output_type=tf.int32)),tf.float32)
accuracy3 = tf.reduce_mean(correct_pre3)

accuracy = tf.reduce_mean(correct_pre0*correct_pre1*correct_pre2*correct_pre3)

In [7]:
saver = tf.train.Saver(max_to_keep=3)

CHECK_POINT_DIR = "./captcha/model/"

latest_checkpoint = tf.train.latest_checkpoint(CHECK_POINT_DIR)
update_lr = tf.assign(lr,lr*0.75)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    if latest_checkpoint != None:
        saver.restore(sess,latest_checkpoint)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)
    i_epoch = 0

    for i in range(LOOP_TIMES):
        b_image, b_label0, b_label1, b_label2, b_label3 = sess.run(
            [image_batch_tensor, label0_batch_tensor, label1_batch_tensor, label2_batch_tensor, label3_batch_tensor])
        sess.run(optimizer, feed_dict={x:b_image,
                                       y0:b_label0,
                                       y1:b_label1,
                                       y2:b_label2,
                                       y3:b_label3})
        i_epoch_new = i//(TRAIN_NUM/BATCH_SIZE) + 1
        if i_epoch != i_epoch_new:
            i_epoch = i_epoch_new
            if i_epoch%5 == 0:
                sess.run(update_lr)

        if i%100 == 0:
            acc0, acc1, acc2, acc3,acc, loss_ = sess.run([accuracy0, accuracy1,accuracy2, accuracy3,accuracy,total_loss],
                                                     feed_dict={x:b_image,
                                                               y0:b_label0,
                                                               y1:b_label1,
                                                               y2:b_label2,
                                                               y3:b_label3})
            learning_rate = sess.run(lr)
            print("Iter:%d/%d epoch:%d,    Loss:%.3f    Accuracy:%.2f,%.2f,%.2f,%.2f     Total_Acc =%.4f   Learning_rate:%.8f" % (
                i, LOOP_TIMES, i_epoch, loss_, acc0, acc1, acc2, acc3, acc, learning_rate))
        if (i+1) % 2000 ==0:
            print("save model.   at {} steps".format(LOOP_TIMES))
            saver.save(sess, CHECK_POINT_DIR + '/crack_captcha.model',global_step=LOOP_TIMES)
            
    print("Save model at last.")
    saver.save(sess, CHECK_POINT_DIR + '/crack_captcha.model',global_step=LOOP_TIMES)
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Iter:0/7014 epoch:1,    Loss:2.677    Accuracy:0.10,0.11,0.11,0.09     Total_Acc =0.0000   Learning_rate:0.00010000
Iter:100/7014 epoch:1,    Loss:2.706    Accuracy:0.08,0.05,0.06,0.11     Total_Acc =0.0000   Learning_rate:0.00010000
Iter:200/7014 epoch:2,    Loss:2.707    Accuracy:0.05,0.09,0.09,0.09     Total_Acc =0.0000   Learning_rate:0.00010000
Iter:300/7014 epoch:2,    Loss:2.709    Accuracy:0.06,0.06,0.06,0.05     Total_Acc =0.0000   Learning_rate:0.00010000
Iter:400/7014 epoch:3,    Loss:2.682    Accuracy:0.08,0.12,0.11,0.09     Total_Acc =0.0000   Learning_rate:0.00010000
Iter:500/7014 epoch:3,    Loss:2.344    Accuracy:0.25,0.19,0.17,0.19     Total_Acc =0.0039   Learning_rate:0.00010000
Iter:600/7014 epoch:4,    Loss:1.873    Accuracy:0.46,0.24,0.27,0.45     Total_Acc =0.0078   Learning_rate:0.00010000
Iter:700/7014 epoch:4,    Loss:1.435    Accuracy:0.58,0.38,0.38,0.46     Total_Acc =0.0547   

Iter:6600/7014 epoch:38,    Loss:0.146    Accuracy:0.96,0.92,0.95,0.95     Total_Acc =0.8008   Learning_rate:0.00001335
Iter:6700/7014 epoch:39,    Loss:0.149    Accuracy:0.98,0.94,0.93,0.95     Total_Acc =0.8359   Learning_rate:0.00001335
Iter:6800/7014 epoch:39,    Loss:0.111    Accuracy:0.96,0.97,0.95,0.98     Total_Acc =0.8516   Learning_rate:0.00001335
Iter:6900/7014 epoch:40,    Loss:0.119    Accuracy:0.97,0.95,0.96,0.97     Total_Acc =0.8633   Learning_rate:0.00001001
Iter:7000/7014 epoch:40,    Loss:0.124    Accuracy:0.96,0.94,0.96,0.98     Total_Acc =0.8594   Learning_rate:0.00001001
Save model at last.
